In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas_ta as ta
import numpy as np
from core.data_sources import CLOBDataSource


In [ ]:
clob = CLOBDataSource()
CONNECTOR_NAME = "binance"
TRADING_PAIR = "ZEC-USDT"
INTERVAL = "1m"
DAYS = 15  

clob.load_candles_cache()
candles = next(
    (c for c in clob.candles_cache.values()
     if c.trading_pair == TRADING_PAIR
     and c.connector_name == CONNECTOR_NAME
     and c.interval == INTERVAL),
    None
)

if not candles:
    candles = await clob.get_candles_last_days(CONNECTOR_NAME, TRADING_PAIR, INTERVAL, DAYS, from_trades=False)
    clob.dump_candles_cache()

candles_df = candles.data

In [ ]:
ema_fast_period = 25
ema_slow_period = 50
adx_period = 14
volume_period = 20
volume_multiplier = 1.5

In [ ]:
candles_df.ta.ema(length=ema_fast_period, append=True)
candles_df.ta.ema(length=ema_slow_period, append=True)
candles_df.ta.adx(length=adx_period, append=True)

# rolling average volume
candles_df["avg_volume"] = candles_df["volume"].rolling(window=volume_period).mean()

# Column names generated by pandas_ta
ema_fast_col = f"EMA_{ema_fast_period}"
ema_slow_col = f"EMA_{ema_slow_period}"
adx_col = f"ADX_{adx_period}"

In [ ]:
candles_df["signal"] = 0

# Long condition: fast EMA crosses above slow EMA, volume > avg * 1.5, ADX > 25
long_condition = (
    (candles_df[ema_fast_col] > candles_df[ema_slow_col]) &
    (candles_df[ema_fast_col].shift(1) <= candles_df[ema_slow_col].shift(1)) &
    (candles_df["volume"] > candles_df["avg_volume"] * volume_multiplier) &
    (candles_df[adx_col] > 25)
)

# Short condition: fast EMA crosses below slow EMA, volume > avg * 1.5, ADX > 25
short_condition = (
    (candles_df[ema_fast_col] < candles_df[ema_slow_col]) &
    (candles_df[ema_fast_col].shift(1) >= candles_df[ema_slow_col].shift(1)) &
    (candles_df["volume"] > candles_df["avg_volume"] * volume_multiplier) &
    (candles_df[adx_col] > 25)
)

candles_df.loc[long_condition, "signal"] = 1
candles_df.loc[short_condition, "signal"] = -1

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True, vertical_spacing=0.02,
    subplot_titles=('OHLC with EMAs', 'ADX', 'Signal'),
    row_heights=[0.6, 0.2, 0.2]
)

# Candlestick
fig.add_trace(
    go.Candlestick(
        x=candles_df.index,
        open=candles_df['open'],
        high=candles_df['high'],
        low=candles_df['low'],
        close=candles_df['close'],
        name='Candlesticks'
    ), row=1, col=1
)

# EMAs
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[ema_fast_col],
                         line=dict(color='cyan', width=1.5),
                         name=f'EMA {ema_fast_period}'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[ema_slow_col],
                         line=dict(color='orange', width=1.5),
                         name=f'EMA {ema_slow_period}'), row=1, col=1)

# ADX
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[adx_col],
                         line=dict(color='lime', width=1.5),
                         name=f'ADX {adx_period}'), row=2, col=1)
fig.add_hline(y=25, line_dash="dot", line_color="red", row=2, col=1)

# Signal line
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df['signal'],
                         mode='lines',
                         name='Signal',
                         line=dict(color="white", width=1.5)),
              row=3, col=1)


In [ ]:
fig.update_layout(
    title=f'{CONNECTOR_NAME} - {TRADING_PAIR} - {INTERVAL} (EMA Crossover)',
    width=1200, height=800,
    font=dict(color='#e1e1e1'),
    plot_bgcolor='#1e1e1e',
    paper_bgcolor='#1e1e1e',
    xaxis_rangeslider_visible=False,
    legend=dict(bgcolor='rgba(0,0,0,0)'),
    yaxis=dict(title='Price'),
    yaxis2=dict(title='ADX', showgrid=False),
    yaxis3=dict(title='Signal', showgrid=False),
    showlegend=False
)

fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)

In [ ]:
fig.show()

In [ ]:
candles_df[["close", ema_fast_col, ema_slow_col, adx_col, "signal"]].tail(10)

### EMA Crossover Strategy

Implemented an EMA (Exponential Moving Average) Crossover strategy. The approach identifies trend shifts based on the crossover between a short-term and a long-term EMA, providing clear buy and sell signals during momentum changes.

## Features:
1. Short-term EMA (e.g., 9-period) for quick trend detection

2. Long-term EMA (e.g., 21-period) for overall trend direction

3. Crossover logic, enter long when the short-term EMA crosses    above the long-term EMA, and enter short on the reverse

4. RSI (14) as a momentum filter to confirm trade entries

Check the controller file `ema_crossover.py` in `app\controllers\directional_trading` for controller implementation